### **Model test results for left edge at 10% miscoverage (90% coverqge)**

In [3]:
import numpy as np
import math
import pandas as pd

In [13]:
df = pd.read_pickle("/content/Calibration_df_run5")
df_test = pd.read_pickle("/content/Test_df_run5")

#### Conformal prediction procedure

**left edge**

In [6]:
# Ordinary CP
cp_left_conf = df['lpred_nonconformity']
cp_left_conf = cp_left_conf.sort_values(ascending=False)
significance_index = math.floor(0.1*(len(cp_left_conf) + 1))
cp_left_conf.iloc[significance_index]

0.06761092398009694

In [7]:
check = 0
interval_len = []
ratio = []


for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']
  pred = df_test.iloc[i]['ledge_pred_sigma']
  left = (pred-cp_left_conf.iloc[significance_index])
  right = (pred+cp_left_conf.iloc[significance_index])

  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.9022569444444445
ave interval: 0.13522184796019399
min interval: 0.13522184796019387
max interval: 0.1352218479601941
min ratio: 0.06483432934606903
max ratio: 0.48689692089832703


#### Normalized Conformal Prediction

In [19]:
df_normcp = pd.read_pickle("/content/Calibration_df_normcp")
df_normcp_test = pd.read_pickle("/content/Test_df_normcp")

df['norm_log_left_error'] = df_normcp['norm_log_left_error']
df['norm_log_right_error'] = df_normcp['norm_log_right_error']
df_test['norm_log_left_error'] = df_normcp_test['norm_log_left_error']
df_test['norm_log_right_error'] = df_normcp_test['norm_log_right_error']

**left edge**

In [21]:
cp_left_conf = df['lpred_nonconformity']/np.exp(-df['norm_log_left_error'])
cp_left_conf = cp_left_conf.sort_values(ascending=False)

significance_index = math.floor(0.1*(len(cp_left_conf) + 1))

cp_left_conf.iloc[significance_index]

3.3610438402400327

In [22]:
check = 0
interval_len = []
ratio = []


for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']
  pred = df_test.iloc[i]['ledge_pred_sigma']
  delta = cp_left_conf.iloc[significance_index]*np.exp(-df_test['norm_log_left_error']).iloc[i]
  left = (pred-delta)
  right = (pred+delta)
  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.9015625
ave interval: 0.15313800025421506
min interval: 0.006111384332013747
max interval: 1.5877470834430225
min ratio: 0.007506539590741063
max ratio: 2.716138148000039


#### Quantile Regression

**left edge**

In [30]:
check = 0
interval_len = []
ratio = []

for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']
  left = df_test.iloc[i]['lower_left_LER']
  right = df_test.iloc[i]['upper_left_LER']
  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.859375
ave interval: 0.1966302084038034
min interval: 0.0333305299282074
max interval: 0.496057391166687
min ratio: 0.05649771946946399
max ratio: 0.5213143580386702


#### Conformal Quantile Regression

**left edge**

In [31]:
left_E_conf1 = (df['lower_left_LER']-df['i_leftline_sigma'])
left_E_conf2 = (df['i_leftline_sigma']-df['upper_left_LER'])
left_E_conf = pd.DataFrame([left_E_conf1, left_E_conf2]).max()

In [32]:
conf = left_E_conf
# sort in ascending order
conf = conf.sort_values(ascending=True)
coverage=0.9
significance_index = math.floor( (coverage*(1/len(conf) + 1))*(len(conf) + 1) )

In [33]:
check = 0
interval_len = []
ratio = []

for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']
  lower = df_test.iloc[i]['lower_left_LER']
  upper = df_test.iloc[i]['upper_left_LER']
  left = (lower - conf.iloc[significance_index])
  right = (upper + conf.iloc[significance_index])
  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.8954861111111111
ave interval: 0.21515355664225944
min interval: 0.051853878166663536
max interval: 0.5145807394051429
min ratio: 0.07081121598219244
max ratio: 0.5701810306991829


CQR-r

**left edge**

In [36]:
left_E_conf1 = (df['lower_left_LER']-df['i_leftline_sigma'])/(df['upper_left_LER']-df['lower_left_LER'])
left_E_conf2 = (df['i_leftline_sigma']-df['upper_left_LER'])/(df['upper_left_LER']-df['lower_left_LER'])
left_E_conf = pd.DataFrame([left_E_conf1, left_E_conf2]).max()

In [37]:
conf = left_E_conf
# sort in ascending order
conf = conf.sort_values(ascending=True)
coverage=0.9
significance_index = math.floor( (coverage*(1/len(conf) + 1))*(len(conf) + 1) )

In [38]:
check = 0
interval_len = []
ratio = []

for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']
  lower = df_test.iloc[i]['lower_left_LER']
  upper = df_test.iloc[i]['upper_left_LER']
  left = ((1 + conf.iloc[significance_index])*lower - conf.iloc[significance_index]*upper)
  right = ((1 + conf.iloc[significance_index])*upper - conf.iloc[significance_index]*lower)
  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.8951388888888889
ave interval: 0.22451697217675087
min interval: 0.0380575788495322
max interval: 0.5664099346420082
min ratio: 0.06451041787092052
max ratio: 0.5952489303105722


#### Defining parameters

In [41]:
df_ideal = pd.read_pickle("/content/Calibration_df_ideal")
df_ideal_test = pd.read_pickle("/content/Test_df_ideal")

df['ideal_lower_left'] = df_ideal['ideal_lower_left']
df['ideal_upper_left'] = df_ideal['ideal_upper_left']
df_test['ideal_lower_left'] = df_ideal_test['ideal_lower_left']
df_test['ideal_upper_left'] = df_ideal_test['ideal_upper_left']

In [42]:
check = 0
interval_len = []
ratio = []

for i in range(len(df_test)):
  original = df_test.iloc[i]['i_leftline_sigma']
  left = df_test.iloc[i]['ideal_lower_left']
  right = df_test.iloc[i]['ideal_upper_left']
  interval_len.append(right-left)
  ratio.append((right-left)/original)

  if (original <= right) and (original >= left):
    check += 1
print('coverage: ' + str(check/len(df_test)))
print('ave interval: ' + str(np.mean(interval_len)))
print('min interval: ' + str(min(interval_len)))
print('max interval: ' + str(max(interval_len)))
print('min ratio: ' + str(min(ratio)))
print('max ratio: ' + str(max(ratio)))

coverage: 0.8977430555555556
ave interval: 0.1035607
min interval: [0.03245822]
max interval: [0.23893023]
min ratio: [0.02762939]
max ratio: [0.5012759]
